In [1]:
import json
from openai import OpenAI

from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI
from homeworks.utils.additional import currency_rate, country_population

client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('knowledge')
task = get_task(token, print_task=True)
question = task['question']

----------- Task description -----------
msg: I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model
question: podaj populację Francji
database #1: Currency http://api.nbp.pl/en.html (use table A)
database #2: Knowledge about countries https://restcountries.com/ - field 'population'
-----------    ----------    -----------


In [2]:
# ask LLM about task. Let it give answer as function calling for 3 functions
# 1 - general question; 2- currency; 3 - population
function_descriptions = [
    {
        "name": "general_info",
        "description": "Answering for general information question",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "Copy of user question"
                },
                "answer": {
                    "type": "string",
                    "description": "Answer for user question"
                },
            },
            "required": ["question", "answer"]
        }
    },
    {
        "name": "currency_rate",
        "description": "Answering for question about currency rate",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "a three letter currency code (ISO 4217 standard), for example USD for United States"
                },
            },
            "required": ["code"]
        }
    },
    {
        "name": "country_population",
        "description": "Answering for question about population number",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {
                    "type": "string",
                    "description": "name of country written with small letter, for example for Poland: poland"
                },
            },
            "required": ["country"]
        }
    },
]

In [3]:
# find final answer
response =  client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": question}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",
    timeout=10
)

function_name = response.choices[0].message.function_call.name
function_arguments = json.loads(response.choices[0].message.function_call.arguments)

In [4]:
if function_name == 'country_population':
    print(f'country_population arguments: {function_arguments}')
    answer = country_population(function_arguments['country'])
elif function_name == 'currency_rate':
    print(f'currency_rate arguments: {function_arguments}')
    answer = currency_rate(function_arguments['code'])
else:
    answer = function_arguments['answer']
    print(answer)

send_answer(token, answer)

country_population arguments: {'country': 'france'}
I have value for france: 67391582
Sending answer: done! :)
